In [14]:
!pip install -q datasets accelerate evaluate trl accelerate bitsandbytes peft


In [15]:
import json
from datasets import Dataset

jsonl_path = "/content/100 samples.jsonl"

data = []
with open(jsonl_path, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line:  # ✅ skip empty lines
            data.append(json.loads(line))

dataset = Dataset.from_list(data)

In [16]:
from huggingface_hub import login

# Paste your HF token when prompted (required to access Mistral 7B Instruct v2)
login()


In [9]:
!pip install -U bitsandbytes

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)
model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [18]:
from peft import prepare_model_for_kbit_training
model = prepare_model_for_kbit_training(model)

In [7]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/100 samples.jsonl")  # Replace with your file


In [8]:
dataset = dataset["train"]

In [19]:
def format_and_tokenize(example):
    prompt = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=True
    )
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer(
        prompt,
        truncation=True,
        max_length=1024,
        padding="max_length"
    )

tokenized_dataset = dataset.map(
    format_and_tokenize,
    remove_columns=dataset.column_names,
    batched=False
)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [22]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,  # rank
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


In [23]:
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="mistral-7b-hr-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    num_train_epochs=3,
    bf16=True,
    logging_steps=10,
    save_steps=1,
    save_total_limit=2,
    report_to="tensorboard"
)

# === 5️⃣ SFTTrainer ===
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args
)

# === 6️⃣ Train ===
trainer.train()


Truncating train dataset:   0%|          | 0/99 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


KeyboardInterrupt: 

In [21]:
trainer.save_model("mistral-7b-hr-finetuned-final")
tokenizer.save_pretrained("mistral-7b-hr-finetuned-final")

print("✅ Fine-tuning completed and saved.")

NameError: name 'trainer' is not defined

In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# === 1️⃣ Load tokenizer and model ===
model_path = "/content/mistral-7b-hr-finetuned/checkpoint-2"  # Replace with your actual output dir

tokenizer = AutoTokenizer.from_pretrained(model_path)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    quantization_config=bnb_config
)

model.eval()

# === 2️⃣ Create your chat prompt ===
messages = [
    {"role": "system", "content": "You are HR guide for Softweb Solutions. You need to provide accurate, concise HR policy answers."},
    {"role": "user", "content": "How many PLs per year at Priya Softweb?"}
]

# Apply chat template to get prompt
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True  # adds "<|assistant|>\n"
)

# === 3️⃣ Tokenize prompt ===
inputs = tokenizer(
    prompt,
    return_tensors="pt"
).to("cuda" if torch.cuda.is_available() else "cpu")

# === 4️⃣ Generate response ===
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        do_sample=True
    )

# === 5️⃣ Decode and clean output ===
response = tokenizer.decode(output[0], skip_special_tokens=True)

print("\n=== HR Chatbot Response ===")
print(response)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



=== HR Chatbot Response ===
You are HR guide for Softweb Solutions. You need to provide accurate, concise HR policy answers.

How many PLs per year at Priya Softweb? At Softweb Solutions, the number of Paid Time Off (PTO) days or Personal Leave (PL) days varies and is typically determined by each employee's contract or job level. It's essential to review the specific employment agreement for accurate information regarding an individual's leave entitlement.

In some cases, standard policies may include:
- Full-time employees accrue a certain number of vacation days per year, typically ranging from 10-25 days.
- Sick days or personal days might also be included in the PTO bank.

For more detailed information on your specific situation, please refer to your HR representative or the HR handbook provided during your onboarding process.


In [ ]:
messages = [
    {"role": "system", "content": "You are HR guide for Softweb Solutions. You need to provide accurate, concise HR policy answers."},
    {"role": "user", "content": "How many PLs per year at Priya Softweb?"}
]

# Apply chat template to get prompt
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True  # adds "<|assistant|>\n"
)

# === 3️⃣ Tokenize prompt ===
inputs = tokenizer(
    prompt,
    return_tensors="pt"
).to("cuda" if torch.cuda.is_available() else "cpu")

# === 4️⃣ Generate response ===
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        do_sample=True
    )

# === 5️⃣ Decode and clean output ===
response = tokenizer.decode(output[0], skip_special_tokens=True)

print("\n=== HR Chatbot Response ===")
print(response)
